# DSSS: Assessing London Housing Prices

I aim to answer the following research questions:

What are the main factors that influence property values in London according to a random forest model based on London census data?
How do property values vary across London opportunity areas and adjacent areas using a regression discontinuity design?
My main data source will be the 2021 census, which provides comprehensive data on London’s population characteristics such as age, sex, ethnicity, religion, education, occupation, income, housing tenure, and so on.

My outcome variable will be median property value per LSOA, from HMRC price paid data. The random forest technique will be applied to build a model for property value based on the independent variables. The regression discontinuity design will be applied to compare property values between opportunity areas (treatment group) and adjacent areas (control group) using distance from opportunity area boundary as the assignment variable. The assumption is that properties close to the boundary are similar in other aspects except for their location within or outside an opportunity area. The causal effect of being in an opportunity area on property value will be estimated by fitting local linear regressions on both sides of the boundary.



Introduction 

Literature review 

Research question 

Presentation of data 

## Data Importing

In [2]:
pip install UKCensusAPI

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for UKCensusAPI: filename=ukcensusapi-1.1.6-py3-none-any.whl size=27322 sha256=cc597c9ae799583e807f240ada0b7c3a1bcab4c39cf043a6b396b8840fede49a
  Stored in directory: /home/jovyan/.cache/pip/wheels/7f/a8/d8/f73537776580f86ab590e8878518b32ad041b091238827602f
Successfully built UKCensusAPI
Note: you may need to restart the kernel to use updated packages.


In [29]:
import ukcensusapi.Nomisweb as census_api
import numpy as np
import pandas as pd
import os

In [115]:
hh = pd.read_excel("data/tenure - households.xlsx", sheet_name = 3)

In [116]:
hh.head()

,LSOA code,local authority code,local authority name,All Households,Owned outright,Owned with a mortgage or loan,Shared ownership,Rented from Local Authority,Other social rented,Private landlord or letting agency,Other private rented,Rent free
0,E01000001,E09000001,City of London,843,338,144,3,17,4,283,53,1
1,E01000002,E09000001,City of London,825,283,149,2,23,6,324,38,0
2,E01000003,E09000001,City of London,1015,232,145,0,249,52,293,42,2
3,E01000005,E09000001,City of London,482,19,22,1,148,186,91,15,0
4,E01032739,E09000001,City of London,882,118,98,3,3,8,555,97,0


In [119]:
lsoas = hh['LSOA code'].values.tolist()

In [136]:
delimiter = ','
lsoas_string = delimiter.join(lsoas)
lsoas_string = repr(lsoas_string).strip("'")
lsoas_string

'E01000001,E01000002,E01000003,E01000005,E01032739,E01032740,E01000006,E01000007,E01000008,E01000009,E01000011,E01000012,E01000013,E01000014,E01000015,E01000016,E01000017,E01000018,E01000019,E01000020,E01000021,E01000022,E01000024,E01000025,E01000027,E01000028,E01000029,E01000030,E01000031,E01000032,E01000033,E01000034,E01000035,E01000036,E01000037,E01000038,E01000039,E01000040,E01000041,E01000042,E01000043,E01000044,E01000045,E01000046,E01000049,E01000050,E01000051,E01000052,E01000053,E01000054,E01000055,E01000056,E01000057,E01000058,E01000059,E01000060,E01000061,E01000062,E01000063,E01000064,E01000065,E01000066,E01000067,E01000068,E01000069,E01000070,E01000071,E01000072,E01000073,E01000074,E01000075,E01000076,E01000077,E01000078,E01000079,E01000080,E01000081,E01000082,E01000083,E01000084,E01000085,E01000086,E01000087,E01000088,E01000089,E01000090,E01000091,E01000093,E01000094,E01000095,E01000096,E01000097,E01000098,E01000099,E01000100,E01000101,E01000102,E01000103,E01000104,E01000105

In [27]:
api_key = "0xc69d140847c615b394009719198254136d8de74d"

In [ ]:
https://www.nomisweb.co.uk/api/v01/dataset/NM_2021_1.data.csv?date=latest&geography=633339927...633339932,633340006...633340009,633339934,633339937...633339939,633339961,633339997,633339998,633340010,633372412,633372413,633339933,633339958...633339960,633339962,633339963,633339976,633339978...633339981,633339999,633339923...633339926,633339975,633372414,633372415,633339917...633339920,633339977,633339935,633339936,633340004,633340005,633339964,633339965,633339967...633339969,633339955,633339971...633339974,633339915,633339916,633339957,633339982...633339984,633340000...633340003,633339910,633339911,633339914,633372416,633372417,633339909,633339912,633339913,633339944,633339966,633339941,633339942,633339945,633339946,633339970,633339952...633339954,633339956,633339992,633339940,633339943,633339948,633339950,633372418...633372421,633339951,633339985...633339988,633339947,633339949,633371579,633372422,633339989...633339991,633339993,633339921,633339922,633339994...633339996,633370825,633340140...633340142,633340146,633340183,633340184,633340059,633340138,633340139,633340143,633340144,633340058,633340060,633340061,633340064,633340176,633340178...633340180,633340182,633340185,633340158...633340161,633340062,633340063,633340065...633340067,633340120,633340145,633340170,633340171,633340177,633340181,633340162...633340164,633340172,633340173,633340011,633340012,633340015,633340016,633340018,633340013,633340014,633340017,633340019,633340020,633340156,633340157,633340165,633340211,633340213,633340166,633340168,633340174,633340175,633340212,633340087,633340121...633340125,633340078...633340083,633340050...633340054,633340215,633340148,633340150,633340154,633340155,633340147,633340149,633340153,633371857,633371858,633340084...633340086,633340118,633340119,633340190,633340191,633340193,633340208,633340209,633340214,633340167,633340169,633340192,633340194,633340195,633340025,633340027,633340028,633340126,633340127,633340048,633340049,633340055...633340057,633340206,633340089,633340186,633340187,633340189,633340022...633340024,633340026,633340029,633371859,633371860,633340090,633340091,633340096,633340151,633340152,633340021,633340044...633340047,633340068,633340070,633340076,633340188,633340207,633340210,633340088,633340092...633340095,633340069,633340071...633340074,633340077,633340131...633340133,633340200,633340129,633340134...633340137,633340205,633340075,633340098...633340100,633340102,633340105,633340128,633340130,633340201,633340202,633340097,633340101,633340103,633340104,633340108,633340109,633340043,633340196,633340197,633340199,633340203,633340204,633340040,633340107,633340110...633340112,633340117,633340030,633340031,633340039,633340041,633340106,633340036,633340113,633340115,633340198,633340033,633340037,633340114,633340116,633340032,633340034,633340035,633340038,633371861...633371866,633340042,633371867...633371871,633340354...633340357,633340305,633340307,633340311,633340352,633340353,633340358,633340224,633340226...633340229,633340292...633340294,633340297,633340223,633340291,633340295,633340296,633340225,633340306,633340308,633340309,633340330,633340325...633340329,633340264,633340318...633340322,633371735,633371736,633340247,633340248,633340250,633340310,633340338...633340340,633340258,633340259,633340263,633340270,633340323,633340244...633340246,633340249,633340324,633340290,633340341...633340344,633340216,633340217,633340260...633340262,633340251,633340253,633340256,633340257,633340284,633340278,633340282,633340298,633340300,633340303,633340304,633340218...633340222,633340231,633340285,633340286,633340299,633340301,633340302,633340265...633340269,633371737,633371738,633340239,633340240,633340252,633340254,633340279,633340280,633340283,633340237,633340238,633340241,633340243,633340281,633340232...633340234,633340236,633340348,633340255,633340332,633340335...633340337,633340242,633340331,633340333,633340334,633340349,633340230,633340235,633340312,633340314,633340351,633340313,633340315,633340316,633340345,633340350,633340272,633340274,633340277,633340317,633340346,633340347,633340271,633340273,633340275,633340276,633340287...633340289,633370812,633340469...633340471,633340473...633340475,633340398,633340399,633340401,633340405,633371875,633371876,633340400,633340402,633340425,633340472,633371499,633340420...633340424,633340403,633340404,633340500,633340501,633340506,633340368,633340371...633340374,633340454...633340457,633340459,633340444,633340445,633340448,633340450,633340451,633340367,633340369,633340370,633340502,633371877,633371878,633340384,633340499,633340503,633371879,633371880,633340383,633340385...633340388,633340452,633340453,633340458,633340460,633340446,633340447,633340485,633340486,633340491,633340391,633340394...633340396,633340504,633340505,633340435...633340440,633340449,633340484,633340487,633340489,633340389,633340390,633340392,633340397,633340441,633340488,633340490,633340508,633340510,633340511,633340493,633340497,633340498,633340512,633371881,633371882,633340477,633340479...633340483,633340375,633340513,633340515,633340518,633340520,633340377,633340381,633340382,633340442,633340443,633340514,633340516,633340517,633340519,633340393,633340406,633340407,633340411...633340413,633340359,633340362,633340364,633340365,633340507,633340476,633340478,633371492,633371883...633371885,633340427...633340431,633340360,633340361,633340363,633340366,633340509,633340376,633340378...633340380,633340463,633340408...633340410,633340417,633340419,633371491,633371515,633340461,633340462,633340464,633340465,633340468,633340414...633340416,633340418,633340466,633340467,633340426,633340432...633340434,633371493,633371886...633371888,633340492,633340494...633340496,633340651...633340655,633340657,633340563...633340566,633340656,633340671,633340674...633340677,633340614,633340615,633340618,633340619,633372326,633372327,633340583...633340585,633340588,633340592,633340521,633340525,633340568...633340571,633340549,633340550,633340691...633340695,633340616,633340617,633340668,633340670,633340672,633340678,633340567,633340572,633340610,633340611,633340547,633340548,633340590,633340591,633340698,633340586,633340587,633340589,633340643,633340645,633340574,633340576,633340577,633340580,633340669,633340603,633340604,633340609,633340612,633340613,633340573,633340575,633340578,633340579,633340673,633340522...633340524,633340529,633340552,633340545,633340551,633340635,633340699,633372328,633372329,633340593...633340595,633340598,633340602,633340527,633340543,633340544,633340546,633340553,633340696,633340697,633340700,633340701,633340703,633340526,633340528,633340679,633340681,633340596,633340599,633340605...633340607,633340642,633340646,633340648...633340650,633340680,633340682,633340683,633340685,633340687,633340536...633340539,633340636,633340637,633340639,633340640,633340704,633340601,633340608,633340662,633340684,633340686,633340597,633340600,633340659,633340660,633340663,633340702,633340705...633340711,633340626...633340628,633340630,633340631,633340658,633340661,633340664,633340665,633340667,633340540...633340542,633340629,633340632...633340634,633340638,633340641,633340555...633340558,633340666,633340622...633340625,633340554,633340559...633340562,633340530...633340533,633340535,633340688...633340690,633370808,633370809,633340581,633340582,633340644,633340647,633370814,633340534,633340620,633340621,633370816,633340773...633340775,633340778,633340779,633340759...633340762,633340765,633340776,633340777,633340787,633340789,633340794,633340744...633340746,633340750,633340751,633340737...633340740,633340743,633340752...633340754,633340758,633340769,633340755...633340757,633340767,633340788,633340793,633340747...633340749,633340763,633340764,633340732,633340736,633340790...633340792,633340830...633340834,633340712,633340714,633340718,633340719,633340766,633340768,633340770...633340772,633340741,633340742,633340797,633340798,633340835,633340713,633340715...633340717,633340733...633340735,633373605,633340823,633340825...633340827,633340829,633340821,633340822,633340824,633340828,633340725...633340728,633340730,633340724,633340729,633340818...633340820,633340795,633340796,633340799...633340801,633340731,633340809...633340811,633340814...633340817,633373606,633373607,633340808,633340812,633340813,633373608,633340720,633340721,633373609,633340781...633340783,633340786,633340723,633340780,633340784,633340785,633340722,633340802...633340807,633373610,633339905...633339908,633370963,633370964,633340992,633340993,633341011,633341017,633341019,633340940,633341012,633341014,633341015,633341018,633340939,633340942,633340943,633341016,633340941,633340944...633340947,633341002,633341003,633341006,633341007,633341013,633341001,633341004,633341005,633341008,633341009,633340991,633340994,633340995,633340997,633340999,633340996,633340998,633341000,633341046,633341047,633340854...633340857,633340859,633340862,633340858,633340860,633340863,633340967,633341010,633341029...633341031,633341037...633341039,633340861,633340965,633340968...633340970,633340972,633340852,633341040,633341041,633341043,633341048,633340864,633340971,633341033,633341036,633372085,633372086,633340866,633340867,633341032,633341034,633341035,633340841...633340844,633341042,633340849,633340851,633340986,633340987,633340989,633340865,633340868...633340873,633340966,633340973,633372087...633372089,633340846...633340848,633340853,633340836...633340840,633340845,633340902,633340903,633340905,633340906,633340901,633341020,633341021,633341023,633372090...633372092,633340982,633340983,633340985,633340990,633340918...633340920,633340984,633340988,633341022,633341024,633341026...633341028,633340895...633340899,633340907,633340908,633340910,633340912,633340913,633340892,633340893,633340900,633340951,633340956,633340909,633340911,633340933,633340934,633340938,633340975,633340977...633340980,633340914...633340917,633340921,633340922,633340974,633340981,633340891,633340957,633340958,633340963,633340964,633340932,633340935...633340937,633340929,633340948,633340949,633340952...633340954,633340887...633340890,633340950,633340959,633340961,633340962,633340976,633340925,633340926,633340931,633340955,633340874,633340875,633340882,633340923,633340930,633340924,633340927,633340928,633340960,633340876,633340883...633340886,633340877...633340881,633340850,633341044,633341045,633341049,633370815,633372093...633372097,633340894,633340904,633341025,633372098,633341120,633341122,633341160...633341162,633341158,633341159,633341163...633341169,633341171,633341172,633341121,633341170,633341173,633341174,633341192...633341194,633341196,633341116...633341118,633341123,633341200,633341195,633341197...633341199,633341108,633341175,633341177,633341181,633341183,633341176,633341178...633341180,633341182,633341109,633341113...633341115,633370967,633370968,633341127...633341129,633341131,633341073,633341111,633341112,633341119,633341064,633341066,633341130,633341132,633341072,633341110,633341142...633341144,633341051,633341091,633341098,633341099,633371972,633371973,633341065,633341133,633341134,633341136...633341138,633341145...633341149,633341059...633341063,633341067,633341077,633341079...633341081,633371974,633371975,633341050,633341052...633341054,633341135,633341139...633341141,633341068...633341071,633341212,633341092...633341097,633341055...633341058,633341202,633341210,633341211,633341213...633341216,633341084,633341087,633341201,633341203,633341205,633341101,633341102,633341107,633341233,633341238,633341239,633341074,633341075,633341185,633341188...633341191,633341103...633341106,633341082,633341083,633341085,633341086,633341090,633341236,633341154,633341232,633341234,633341235,633341237,633341204,633341206...633341209,633341226,633341228,633341229,633341231,633341100,633341150,633341156,633341157,633341184,633341186,633341187,633341220,633341151...633341153,633341155,633341225,633341227,633341230,633370818,633341076,633341078,633341124...633341126,633341088,633341089,633371976,633371977,633341218,633341223,633341224,633341217,633341219,633341221,633341222,633341293,633341295,633341391,633341397,633341292,633341390,633341392,633341393,633341396,633341294,633341296...633341298,633371278,633371279,633341258,633341261...633341264,633341257,633341259,633341260,633341362,633341382,633341289,633341291,633341363,633341394,633341395,633341283...633341286,633341288,633341318,633341381,633341385...633341387,633341383,633341384,633341388,633341389,633341306,633341317,633341320,633341321,633341323,633341265,633341267,633341268,633341272,633341290,633341352...633341355,633341359,633341266,633341319,633341322,633341324,633341287,633341356,633371872,633341255,633341256,633341299,633341358,633341361,633341251,633341327,633341333,633341357,633341360,633341364,633341300...633341303,633341325,633341326,633341330,633341350,633341351,633341365,633341367...633341369,633341371,633341269...633341271,633341370,633371873,633371874,633341248...633341250,633341252,633341253,633341304,633341305,633341408,633341411...633341413,633341328,633341331,633341336,633341338,633341339,633341308,633341309,633341311,633341329,633341337,633341332,633341334,633341335,633341340,633371277,633371280,633341254,633341341,633341342,633341344,633341414,633341273,633341275,633341278,633341310,633341313,633341366,633341376...633341378,633341380,633341343,633341346,633341407,633341409,633341410,633341274,633341276,633341277,633341279,633341280,633341312,633341314...633341316,633341345,633341347,633341242,633341373...633341375,633341379,633341281,633341282,633341400,633341402...633341404,633341244,633341245,633341247,633341348,633341401,633341405,633341240,633341241,633341243,633341246,633341349,633341372,633341307,633341398,633341399,633341406,633370820,633341518...633341521,633341516,633341517,633371693...633371695,633371697,633371699,633341419...633341423,633341532...633341534,633372117,633341501...633341504,633341506,633341468,633341469,633341472,633341473,633371696,633371698,633341415...633341418,633341467,633341470,633341471,633341474,633341522...633341524,633341527,633341530,633341531,633341432,633341433,633341435,633341439,633341434,633341436...633341438,633341525,633341526,633341528,633341529,633341425,633341429...633341431,633341507,633341512...633341514,633341480,633341484...633341486,633341508...633341511,633341515,633341424,633341426...633341428,633341479,633341481...633341483,633341449,633341450,633341454,633341455,633341462...633341466,633341448,633341451...633341453,633341456...633341459,633341487...633341489,633341494,633341495,633341460,633341461,633371686,633341441,633341442,633341446,633341447,633341440,633341443...633341445,633341499,633341500,633341505,633370813,633341490...633341493,633372118...633372121,633371684,633371685,633372122...633372126,633341496,633341497,633371692,633372127...633372129,633341475...633341478,633371700,633371688,633371690,633372130...633372136,633341498,633371687,633372137...633372140,633341535,633371689,633371691,633372141,633341619,633341620,633341624,633341631,633341633,633341536,633341621,633341623,633373536,633373537,633341610,633341613,633341615,633341618,633341622,633341546,633341547,633341611,633341632,633341634,633341544,633341545,633341548,633341584,633341585,633341537...633341542,633341563,633341612,633341614,633341617,633341562,633341616,633341637,633341639,633341640,633341581...633341583,633341586,633371665,633371674,633341604...633341608,633341557...633341560,633341598...633341600,633341602,633341561,633341567,633341638,633341641...633341643,633341554,633341597,633341601,633341603,633341565,633341566,633341574...633341576,633341578,633341552,633341555,633341577,633341579,633341654...633341657,633341550,633341551,633341553,633341556,633341646,633341658,633341568,633341580,633341626,633341627,633373538,633373539,633341644,633341648,633341649,633341652,633341653,633341645,633341647,633341650,633341651,633371666,633371673,633341587,633341589,633341590,633341625,633341628,633341629,633341571,633341572,633341588,633341591,633373540...633373543,633341594,633341595,633373544...633373547,633341609,633341636,633371664,633371667,633371671,633341543,633341549,633341630,633341635,633341573,633371662,633371663,633341564,633341569,633341570,633373548,633341593,633341596,633371670,633371672,633341592,633371668,633371669,633341682...633341685,633373383,633373384,633341760,633341762,633341763,633341766,633341759,633341761,633341764,633341765,633341748...633341751,633341670,633341745...633341747,633341666,633341668,633341669,633341671,633341672,633341660...633341662,633341665,633341667,633341673,633341734,633341737,633341702...633341704,633341736,633341659,633341663,633341664,633341677,633341681,633341735,633341738...633341740,633341674...633341676,633341678...633341680,633341700,633341701,633341705...633341707,633341696,633341717,633341720,633341721,633341692,633341715,633341716,633341718,633341719,633341693,633341694,633341697,633341699,633341687...633341691,633341695,633341698,633341724...633341726,633341686,633341708...633341710,633341727...633341731,633341752...633341756,633341758,633341711...633341714,633341742,633370998...633371000,633373385,633373386,633341722,633341723,633341733,633341757,633341732,633341741,633341743,633341744,633341774,633341776,633341779,633341780,633341839,633341840,633341842,633341843,633341767,633341775,633341777,633341778,633341892,633341895,633341898,633341899,633341893,633341894,633341896,633341897,633341900,633341903,633341904,633341907,633341901,633341902,633341905,633341906,633341768...633341771,633341831,633341832,633341836,633341837,633341781...633341784,633341877,633341880,633341882...633341884,633341888,633341890,633341891,633341772,633341773,633341801,633341802,633341878,633341879,633341881,633371889,633371890,633341830,633341833...633341835,633341796...633341798,633341800,633341785...633341788,633341885...633341887,633341889,633341818,633341819,633341822,633371229,633371891,633371892,633341824,633341826,633341827,633341829,633341817,633341823,633341825,633341828,633341803,633341804,633341808,633341809,633341870,633341872,633341873,633341876,633341869,633341871,633341874,633341875,633341846,633341848,633341849,633341852,633341845,633341847,633341850,633341851,633341820,633341821,633341862,633341863,633341853...633341855,633341858,633341799,633341813,633341815,633341816,633341805...633341807,633341861,633341856,633341857,633341859,633341860,633341789,633341790,633341792,633341794,633341791,633341793,633341795,633341865,633341810...633341812,633341814,633341864,633341866...633341868,633341838,633341841,633341844,633370821,633341914,633341916,633342021,633342022,633341915,633341917...633341919,633372532,633372533,633342023...633342026,633341944,633341946,633341947,633341949,633341940...633341943,633341945,633341939,633341948,633341950...633341952,633341920...633341924,633341908,633341910...633341913,633341983...633341985,633341988,633341925,633341926,633341996,633341998,633342001,633342028...633342032,633341909,633341995,633341997,633341999,633342000,633341978,633341981,633341982,633342027,633372534,633372535,633341955,633341961,633341962,633372536,633372537,633341986,633341987,633341990,633341991,633341964...633341968,633341953,633341954,633341956,633341957,633341928,633341930,633341979,633341980,633341958...633341960,633341963,633341989,633341992...633341994,633341927,633341929,633341931,633372538...633372541,633342033,633342034,633342036,633342038,633342005,633342006,633342035,633342037,633341933,633341936,633341938,633372542,633372543,633342008,633342009,633342012,633342013,633341932,633341934,633341935,633341937,633342014,633342015,633342017,633342018,633342010,633342011,633342016,633342019,633342020,633342002...633342004,633342007,633370822,633341969...633341977,633342071,633342073,633342106,633342107,633342072,633342074,633342076,633342078,633342093,633342095,633342097,633342098,633342100,633342101,633342075,633342077,633342079,633342091,633342108,633342110,633342096,633342099,633342102,633342120,633342125,633342103...633342105,633342109,633342088,633342090,633342092,633342094,633371730,633371731,633342127,633342128,633342130,633342133,633342135,633342121,633342122,633342126,633342129,633342132,633342134,633342063,633342064,633342089,633342170,633342171,633342176,633342040,633342041,633342119,633342123,633342124,633342131,633342148,633342169,633342172...633342174,633342039,633342147,633342150,633370965,633370966,633342067,633342069,633342111,633342112,633342149,633342175,633342065,633342066,633342068,633342070,633342113,633342144...633342146,633342151,633342042...633342046,633371732...633371734,633342047,633342051,633342053,633342054,633342184,633342048,633342049,633342052,633342177,633342152...633342154,633342156,633342160,633342114...633342118,633342050,633342084,633342087,633342155,633342179,633342082,633342086,633342157...633342159,633342178,633342180...633342183,633342060,633342080,633342081,633342083,633342085,633342055...633342059,633342061,633342062,633342165...633342167,633342161...633342164,633342168,633342137,633342138,633342142,633342143,633342136,633342139...633342141,633342264,633342266...633342268,633342273,633342263,633342265,633342269,633342274...633342276,633342229...633342233,633342224,633342227,633342270...633342272,633342316,633342317,633342321,633342322,633342209,633342210,633342222,633342223,633342228,633342221,633342225,633342226,633342261,633342318,633342207,633342208,633342211,633342212,633342262,633342256,633342258,633342285,633342319,633342320,633342206,633342257,633342259,633342260,633342286,633342249...633342251,633342253,633342255,633342252,633342254,633342300,633342301,633342303,633342241...633342245,633342205,633342302,633372544...633372548,633342306...633342308,633372549,633372550,633342201,633342203,633342247,633342298,633342299,633342213...633342216,633342218,633342219,633342199,633342200,633342220,633342246,633342248,633342185,633342186,633342323,633342325,633342326,633342187,633342188,633342190...633342192,633342217,633342202,633342204,633342304,633342305,633342333,633372551,633372552,633342324,633342327...633342330,633342193...633342197,633342331,633342332,633342334...633342336,633342189,633342291,633342292,633342294,633342297,633342198,633342293,633342295,633342296,633372553...633372556,633342277,633342278,633342310,633342312,633372557,633372558,633342236,633342309,633342313...633342315,633342279...633342281,633342283,633372559,633372560,633342235,633342237,633342238,633342311,633342234,633342239,633342240,633342282,633342284,633342287...633342290,633370823,633342349,633342350,633342353...633342355,633342399,633342404...633342406,633342410,633342412,633342413,633342416,633342417,633342345,633342347,633342348,633342458,633371978,633371979,633342351,633342352,633342357,633342358,633342460,633342461,633342356,633342359...633342362,633342450,633342453...633342455,633342398,633342401,633342411,633342414,633342415,633342397,633342400,633342402,633342407...633342409,633342418,633342419,633342449,633342451,633342363...633342366,633342369,633342443,633342444,633342456,633342457,633371980,633371981,633342421,633342422,633342434,633342452,633371982,633371983,633342367,633342368,633342436,633342437,633342440,633342403,633342423,633342438,633342439,633371984,633371985,633342420,633342425,633342433,633371227,633371228,633371986,633371987,633342429...633342432,633342435,633342442,633342445...633342448,633342424,633342426...633342428,633342441,633342337,633342338,633342370,633342372,633342375,633342339...633342343,633342371,633342373,633342374,633342390...633342392,633342377...633342379,633342382,633371162,633371180,633342383...633342386,633342394,633342376,633342380,633342381,633342393,633342395,633342387...633342389,633342396,633342459,633342462...633342464,633370819,633342344,633371988...633371990,633342346,633371225,633371226,633371991,633371992,633342522,633342523,633342525,633342569,633342572,633342576,633342520,633342521,633342524,633342526,633342527,633342538,633342504,633342570,633342571,633342573...633342575,633342534,633342536,633342537,633342539,633342540,633342501,633342503,633342505,633342506,633342518,633342511,633342515...633342517,633371522,633371523,633342500,633342502,633342529,633373549,633373550,633342535,633342551,633342554,633342555,633342507,633342510,633342512,633342513,633342519,633342530,633342549,633342550,633342552,633342553,633342528,633342533,633371516...633371518,633371521,633373551,633373552,633342543...633342545,633342547,633342548,633342508,633342509,633342514,633342561,633342562,633342486,633342488,633342491,633342541,633342542,633342546,633342478,633342482,633342483,633342531,633342532,633342487,633342489,633342490,633342556,633342557,633342559,633342469...633342471,633342484,633342558,633342492,633342563...633342566,633342465,633342468,633342479,633342480,633342485,633342467,633342560,633342567,633342568,633373553,633373554,633342466,633342481,633342497...633342499,633342477,633342493...633342496,633371519,633371520,633342472...633342476,633342614...633342617,633342619,633342665...633342668,633342638...633342640,633342669,633342670,633342592...633342595,633342618,633342632,633342634,633342641...633342648,633342596,633342633,633342635...633342637,633342587...633342591,633342626,633342628...633342631,633342649,633342651...633342654,633342578,633342580...633342582,633342627,633342583,633342584,633342621,633342622,633342625,633342577,633342597,633342601,633342612,633342650,633342585,633342586,633342620,633342623,633342624,633342579,633342609,633342610,633342613,633342598...633342600,633373611,633342608,633342611,633342655,633342656,633342659...633342664,633342606,633342607,633342671,633342672,633342675,633342657,633342658,633342673,633373612,633342602...633342605,633342674,633342766...633342770,633342712,633342713,633342715,633342716,633342719,633342696,633342698,633342711,633342714,633342717,633342694,633342695,633342697,633342771,633371570,633372519,633372520,633342724,633342729,633342731,633342732,633342734,633342693,633342720,633342722,633342730,633342733,633342690...633342692,633342723,633342745,633342688,633342689,633342718,633342721,633342746,633342725,633342727,633342728,633342750,633342751,633342683,633342687,633342726,633342752,633342682,633342747...633342749,633342735,633342737,633342740,633342741,633342744,633342684,633342686,633342753,633342755,633342759,633342676...633342678,633342681,633342685,633342736,633342738,633342739,633342742,633342743,633342754,633342756...633342758,633342762,633342679,633342680,633342760,633342761,633342763,633342699,633342701,633342702,633342764,633342765,633342704...633342707,633342700,633342703,633342708...633342710,633342774,633342866...633342869,633342860,633342863...633342865,633371808,633371809,633342861,633342862,633342879,633342886,633371315,633371810...633371812,633342857...633342859,633342937,633342940,633342856,633342878,633342880...633342882,633342938,633342941...633342944,633342848,633342850,633342851,633342883,633342885,633342805,633342807,633342834,633342939,633342945,633342847,633342852,633342853,633342855,633342884,633342804,633342806,633342808,633342818...633342820,633342793...633342795,633342849,633342854,633342796...633342799,633342801,633342831,633342832,633342835...633342837,633342778,633342812,633342813,633342815,633342817,633342779,633342809,633342811,633342814,633342933,633342788,633342791,633342800,633342816,633342802,633342803,633342810,633342830,633342833,633342785...633342787,633342789,633342792,633342777,633342928,633342930,633342934,633342935,633342780...633342784,633342893,633342894,633342916,633342917,633342919,633342790,633342913...633342915,633342918,633342920,633342889,633342929,633342931,633342932,633342936,633342921,633342922,633342925...633342927,633342887,633342888,633342892,633342895,633342908,633342838,633342839,633342890,633342891,633342909,633342823,633342825,633342840,633342923,633342924,633342873...633342875,633342910,633342912,633342842...633342844,633342906,633342911,633342824,633342827,633342841,633342845,633342846,633342870,633342871,633342876,633342877,633342905,633342907,633342821,633342822,633342826,633342828,633342829,633342872,633342898,633342901,633342902,633342904,633342896,633342897,633342899,633342900,633342903,633342772,633342773,633342775,633342776,633370827,633342998,633342999,633343001,633343003,633372330...633372332,633343000,633343002,633343004...633343006,633343047,633343049,633343051,633343052,633343055,633342964...633342966,633342968,633342971,633343086...633343089,633343091,633342956,633342958,633342961...633342963,633342967,633342969,633342970,633371401,633371402,633342955,633342957,633342959,633342960,633343084,633343085,633343090,633343092,633343093,633343026,633343030...633343032,633343035...633343037,633343039,633343025,633343027...633343029,633342981,633342987...633342989,633343033,633343034,633343038,633343040,633343044...633343046,633371399,633371400,633343067,633343068,633343071,633343073,633342982...633342986,633343066,633343069,633343070,633343072,633372333,633372334,633343007,633343008,633343013...633343015,633342973,633342974,633342976,633342978,633342980,633343016,633343021...633343024,633343056,633343057,633343063...633343065,633342972,633342975,633342977,633342979,633343094,633343097,633343099,633343102,633343058...633343062,633343009...633343012,633342950...633342954,633343095,633343096,633343098,633343100,633343101,633343017...633343020,633343077...633343080,633343082,633342946...633342949,633343074...633343076,633343081,633343083,633342991...633342994,633370810,633342990,633342995...633342997,633370811,633343048,633343050,633343053,633343054,633370824,633372335...633372338,633343041...633343043,633371403,633371411,633343219...633343221,633343224,633372521,633372522,633343207,633343209,633343211,633343212,633343147,633343201,633343203,633343222,633343223,633343148...633343151,633343208,633343116...633343118,633343202,633372523,633372524,633343107,633343108,633343204...633343206,633343141...633343145,633343115,633343119,633343140,633343153,633343154,633343103,633343104,633343106,633343130,633343131,633343189...633343192,633343210,633343127,633343129,633343132,633343146,633343170,633343105,633343123,633343124,633343155,633343157,633343133,633343135,633343136,633343139,633343156,633343134,633343138,633343158,633343159,633343182,633343109,633343171...633343173,633343175,633343114,633343128,633343193,633343194,633343218,633343160...633343163,633343181,633343120,633343125,633343126,633343137,633343152,633343176...633343180,633343213...633343217,633343110...633343113,633343168,633343121,633343122,633343184,633343187,633343188,633343174,633343183,633343186,633343195,633343200,633343185,633343196...633343199,633343164...633343167,633343169,633343284,633343285,633343287,633343291,633343317,633343322...633343324,633343319,633343325,633343327,633343331,633343320,633343326,633343328,633343332,633343269,633343316,633343318,633343321,633343286,633343289,633343290,633343298,633343288,633343296,633343302,633343308,633343297,633343299,633343311...633343313,633343292,633343294,633343295,633343300,633343270...633343273,633343275,633343274,633343301,633343304,633343329,633343330,633343351...633343353,633372142,633372143,633343303,633343305...633343307,633343358...633343362,633343293,633343337,633343338,633343368,633343309,633343310,633343314,633343315,633343261...633343263,633343266,633343267,633343233...633343235,633343265,633343364...633343367,633343369,633343339,633343341,633343363,633343370,633343236,633343333,633343340,633372144,633372145,633343276,633343277,633343354...633343356,633343264,633343268,633343278,633343283,633343357,633343237...633343239,633372146,633372147,633343335,633343336,633343343,633343347,633343240,633343242,633343244,633343334,633343342,633343344,633343346,633343348,633343279...633343282,633343243,633343245...633343247,633343249...633343251,633343345,633343228,633343229,633343257,633343259,633343260,633343226,633343231,633343232,633372148...633372150,633343241,633343248,633371577,633372151...633372155,633343225,633343227,633343230,633343254,633343252,633343253,633343255,633343256,633343258,633372156...633372160,633343350,633372161...633372164,633371573,633371578,633372165,633372166,633343349,633371575,633371576,633372167,633371574,633372168...633372174,633343473...633343479,633343450,633343452,633343453,633343456,633343451,633343454,633343455,633343457,633343387,633343388,633343391,633343393,633343389,633343390,633343392,633343429...633343435,633343400...633343406,633343438...633343441,633343487...633343493,633343436,633343437,633343442,633343443,633343379,633343381,633343382,633343384,633343372...633343375,633343502...633343508,633343380,633343383,633343385,633343386,633343371,633343376...633343378,633343517,633343522...633343524,633343480,633343482,633343483,633343486,633343497...633343499,633343501,633343481,633343484,633343485,633372423,633372424,633343494...633343496,633343500,633343509,633343510,633343514,633343515,633343518...633343521,633343511...633343513,633343516,633343466...633343472,633343458,633343459,633343461,633343462,633372425...633372427,633343460,633343463...633343465,633343444...633343449,633370826,633343421...633343428,633343407...633343414,633343396...633343398,633371581,633371583,633343394,633343395,633343399,633343420,633371580,633371582,633343415...633343419,633343525...633343531,633343574,633343575,633343577,633343578,633343587,633343591,633343579...633343585,633343576,633343588...633343590,633343592,633371571,633371572,633343532,633343535...633343537,633343586,633343600...633343602,633343605,633343593,633343595,633343598,633343599,633343549,633343603,633343604,633343606,633343619...633343622,633343632...633343636,633343594,633343596,633343597,633343637,633343533,633343534,633343544,633343545,633343568...633343573,633343609,633343610,633343623...633343631,633343607,633343608,633343611,633343612,633343546...633343548,633343539,633343540,633343617,633343618,633343538,633343541...633343543,633343556...633343561,633343613...633343616,633343562...633343567,633343550...633343555,633343735,633343738,633343759...633343761,633343654,633343658,633343659,633343663,633371813,633371814,633343690,633343732,633343734,633371815...633371818,633343662,633343664...633343666,633343693,633343695,633343736,633343737,633343739,633343740,633343742,633343762...633343766,633343660,633343661,633343667,633371819,633371820,633343692,633343743,633343744,633343748,633343749,633343698,633343699,633343733,633343741,633343682,633343683,633343706,633371821...633371823,633343694,633343745...633343747,633343750,633343704,633343707,633343709,633343710,633370881,633370950,633343685,633343687,633371824...633371826,633343684,633343688,633343689,633371827,633343650,633343686,633343703,633343705,633343708,633343696,633343700...633343702,633343718,633343639,633343721...633343724,633343638,633343643...633343645,633343652,633343646...633343649,633343651,633343653,633343719,633343720,633343773,633371828,633371829,633343697,633343714...633343717,633343640...633343642,633343756,633343758,633343767...633343769,633343771,633343772,633343711,633343712,633343729,633343770,633343774,633343751,633343753...633343755,633343757,633343676,633343678,633343725,633343731,633343752,633343713,633343726,633343727,633343730,633343675,633343677,633343679,633343777,633343778,633343775,633343776,633343779...633343781,633343672,633343673,633343680,633343681,633343728,633343668...633343671,633343674,633343655...633343657,633370828,633370829,633370879,633370880,633370882,633371033,633371830,633343691,633370886,633371006,633371831,633343830,633343887...633343889,633343892,633343828,633343829,633343831,633343833...633343836,633343839...633343841,633343868,633343870,633343873,633343890,633372099,633372100,633343832,633343867,633343869,633343871,633343891,633343822,633343826,633343895,633343896,633343898,633343848...633343851,633343821,633343827,633343893,633343894,633343897,633343899,633343820,633343823,633343824,633343837,633343838,633343875...633343877,633343879,633343880,633343782...633343788,633343842,633343845,633343846,633343853,633372101,633372102,633343807,633343809,633343843,633343844,633343872,633343847,633343852,633343863...633343865,633343803...633343806,633343808,633343816...633343818,633343825,633343861,633343793,633343874,633343878,633343882,633343883,633343856,633343859,633343860,633343862,633343866,633343789,633343792,633343794,633343795,633343886,633343810,633343813,633343881,633343884,633343885,633343796,633343802,633343854,633343855,633343857,633343797,633343798,633343811,633343812,633343858,633343790,633343791,633343814,633343815,633343799...633343801,633343819,633370867,633343920...633343922,633373555...633373557,633343900,633343901,633343903,633373558,633373559,633343923,633343925,633343927...633343929,633343924,633343926,633370977,633370978,633370994,633373560,633373561,633343902,633343904...633343906,633343987,633343990,633373562,633343947...633343949,633343951,633343930...633343933,633373563...633373565,633343988,633343989,633343991,633373566,633373567,633343950,633370995,633370996,633373568,633343909,633343911,633343913,633370997,633343934,633343935,633343952,633343953,633373569...633373571,633343907,633343908,633343910,633343912,633343954...633343956,633373572...633373575,633343982...633343986,633343963,633343965,633343967,633343968,633343994...633343997,633343938,633343940,633343941,633373576,633373577,633343964,633343966,633373578...633373581,633343936,633343937,633343939,633373582,633373583,633343992,633343993,633370979,633373584...633373587,633343975,633343976,633343980,633343981,633343942,633343943,633343946,633373588,633373589,633343944,633343945,633373590...633373593,633343977...633343979,633370980,633370981,633343970,633343972,633343973,633373594,633373595,633343969,633343971,633343974,633343914...633343917,633343957...633343959,633343962,633370986,633370993,633343961,633370982,633370984,633370985,633373596,633373597,633343960,633370983,633370991,633370992,633370987,633373598...633373600,633343918,633370988...633370990,633343919,633373601...633373604,633344020...633344022,633344027,633344028,633344023...633344025,633344029,633344030,633344055...633344057,633344060,633344061,633344026,633344031,633344084,633344086,633344087,633344032,633344085,633344088,633344120,633344121,633344119,633344122...633344125,633344050...633344052,633344054,633344058,633344059,633344014,633344015,633344053,633344089,633344090,633344069...633344071,633344073,633344074,633344016,633344019,633344048,633344049,633344013,633344017,633344018,633344129,633344130,633344067,633344126...633344128,633344131,633344078,633344081,633344132,633344137...633344139,633344062,633344064,633344072,633344075,633372428,633372429,633344083,633344133...633344136,633344077,633344079,633344080,633344082,633344112,633344116,633344063,633344065,633344068,633344113,633344115,633344066,633344092,633344114,633344117,633344118,633344034,633344035,633344037...633344039,633344036,633344076,633344091,633344094,633372430,633372431,633344033,633344105,633344106,633344108,633344111,633344093,633344095...633344098,633344040,633344041,633344043,633344102,633344104,633344004,633344005,633344107,633344109,633344110,633344042,633344044...633344047,633344007,633344099...633344101,633344103,633344006,633344008...633344012,633343998...633344003,633344227,633344231,633344246,633344247,633344206,633344207,633344242,633344248,633344243...633344245,633371239,633371268,633373387,633373388,633344230,633344254...633344256,633344266...633344270,633344202,633344204,633344205,633344208,633344201,633344203,633344219,633344249,633344252,633344221,633344223,633344250,633344251,633344253,633344178,633344265,633344271,633371269,633373389...633373392,633344174,633344300...633344303,633344176,633344177,633344181,633344182,633344218,633344233,633344239...633344241,633344168,633344169,633344171,633344173,633344175,633344220,633344224...633344226,633344281,633344237,633344304...633344307,633344145...633344147,633344217,633344222,633344162,633344180,633344264,633373393,633373394,633344161,633344179,633344282,633344283,633344285,633344289,633344167,633344170,633344172,633344258,633344260,633344291,633344140,633344142,633344144,633344209,633344211,633344290,633344292,633344293,633344295,633344234...633344236,633344238,633344163...633344165,633344284,633344286,633344257,633344259,633344261...633344263,633344141,633344143,633344148,633344151,633344152,633344154,633344210,633344212,633344213,633344216,633344287,633344288,633344294,633344296...633344299,633344149,633344156,633344214,633344215,633344150,633344153,633344155,633344157...633344160,633344166,633344199,633344273,633344274,633344280,633344198,633344200,633344272,633344275...633344279,633344192...633344197,633344183,633344185...633344187,633344190,633344184,633344188,633344189,633344191,633371238,633371267,633373395...633373397,633344228,633344229,633344232,633371240,633373398,633344309,633344311,633344375,633344381,633344308,633344310,633344312,633344361,633344362,633344313,633344376,633344377,633344379,633344380,633344369...633344373,633344329,633344330,633344332...633344334,633344374,633344331,633344357...633344359,633344404,633344353,633344355,633344356,633344360,633373613,633344320,633344321,633344363,633344378,633344328,633344354,633371587,633371589...633371593,633344402,633344403,633344405,633370765,633370766,633344364,633344365,633344367,633344368,633344408,633344318,633344319,633344322...633344324,633344366,633344409,633344410,633373614,633344315...633344317,633344406,633373615,633344335,633344338...633344340,633371584,633371585,633344336,633344337,633344346,633344349,633344352,633344314,633344345,633344347,633344348,633344350,633344351,633344385...633344387,633344407,633344341...633344344,633373616,633344382...633344384,633344393,633344396,633344389,633344392,633344394,633344395,633373617,633373618,633344391,633344397,633344399...633344401,633344326,633344398,633371586,633371588,633373619,633344325,633344327,633344388,633344390,633373620&c2021_restype_3=0...2&measures=20100&uid=0xc69d140847c615b394009719198254136d8de74d

In [90]:
import os

# Define your nomisweb API key
api_key = "0xc69d140847c615b394009719198254136d8de74d"

# Set the environment variable NOMIS_API_KEY
os.environ["NOMIS_API_KEY"] = api_key

# Import ukcensusapi package
from ukcensusapi.Nomisweb import Nomisweb

# Create a Nomisweb object
nomis = Nomisweb("./cache")


In [ ]:
help(Nomisweb)

In [ ]:
nomis.load_metadata(table_name = "KS102EW")

In [138]:
api = CensusApi.Nomisweb("cache")
table = "TS001"
table_internal = "NM_2021_1"
query_params = {}
query_params["date"] = "latest"
query_params["select"] = "GEOGRAPHY_CODE,OBS_VALUE,GEOGRAPHY_NAME"
query_params["C2021_RESTYPE_3"] = "0"
query_params["MEASURES"] = "20100"
query_params['geography'] = 'E01000001,E01000002,E01000003,E01000005'
TS001 = api.get_data(table, query_params)
#api.contextify(table = "TS001", column = "GEOGRAPHY_CODE", table_name = "Population")


In [109]:
TS001.columns

Index(['GEOGRAPHY_CODE', 'OBS_VALUE'], dtype='object')

In [103]:
help(api)

Help on Nomisweb in module ukcensusapi.Nomisweb object:

class Nomisweb(builtins.object)
 |  Nomisweb(cache_dir, verbose=False)
 |  
 |  Nomisweb API methods and data.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cache_dir, verbose=False)
 |      Constructor.
 |      Args:
 |          cache_dir: cache directory
 |      Returns:
 |          an instance.
 |  
 |  contextify(self, table_name, column, table)
 |      Adds context to a column in a table, as a separate column containing the meanings of each numerical value
 |      Args:
 |          table_name: name of census table
 |          column: name of column within the table (containing numeric values)
 |          table:
 |      Returns:
 |          a new table containing an extra column with descriptions of the numeric values.
 |  
 |  get_data(self, table, query_params, r_compat=False)
 |      Downloads or retrieves data given a table and query parameters.
 |      Args:
 |         table: ONS table name, or nomisweb table co

In [91]:

export NOMIS_API_KEY = 0xc69d140847c615b394009719198254136d8de74d

echo 0xc69d140847c615b394009719198254136d8de74d > cache/NOMIS_API_KEY

SyntaxError: invalid syntax (705327269.py, line 1)

Methodology 

Results 

Discussion 

Conclusion 